In [9]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone as PC
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

In [10]:
loader=PyPDFDirectoryLoader("PDFs")

In [11]:
data=loader.load()

In [12]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [13]:
text_chunks=text_splitter.split_documents(data)

In [14]:
from dotenv import load_dotenv
load_dotenv()  

openai_api_key = os.getenv('OPENAI_API_KEY')

In [15]:
embedding=OpenAIEmbeddings()

c:\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [16]:
len(embedding.embed_query("How are you?"))

1536

In [17]:
pinecone_api_key=os.getenv('PINECONE_API_KEY')

In [18]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=pinecone_api_key)


C:\Users\upans\AppData\Roaming\Python\Python311\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [19]:
index_name = "pineconevdb"
pc.create_index(
    name=index_name,
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(
        cloud='aws', 
        region='us-east-1'
    ) 
) 

In [20]:
index = pc.Index(index_name)

In [21]:
docsearch=PC.from_texts([t.page_content for t in text_chunks], embedding, index_name=index_name)

In [22]:
docsearch

In [23]:
query="The Transformer uses multi-head attention in how many ways"

In [24]:
docs=docsearch.similarity_search(query)

In [25]:
docs

[]

In [26]:
llm=OpenAI()

c:\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [27]:
qa=RetrievalQA.from_chain_type(llm=llm, chain_type="stuff",retriever=docsearch.as_retriever())

In [28]:
query="The Transformer uses multi-head attention in how many ways"

In [29]:
qa.run(query)

c:\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' Three different ways.'

In [31]:
import sys
while True:
    user_input=input(f"Input Prompt: ")
    if user_input=='exit':
        print('Exiting')
        sys.exit()
    if user_input=='':
        continue
    result=qa({'query': user_input})
    print(f"Answer: {result['result']}")


Answer: 
NLP stands for Natural Language Processing, which is a field of computer science and artificial intelligence that focuses on enabling computers to understand and process human language in order to perform tasks such as language translation, text summarization, and sentiment analysis. It combines techniques from computer science, linguistics, and machine learning to analyze and extract meaning from large amounts of text data.
Exiting


SystemExit: 

C:\Users\upans\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
